In [1]:
import torch
import torch.nn as nn
import torch.functional as F

%load_ext autoreload
%autoreload 2

In [2]:
from BLM import BLModel

In [3]:
with open('input.txt', 'r', encoding='utf-8') as fp:
    text = fp.read()

    
print(len(text))

1115394


In [4]:
max([len(line) for line in text.splitlines()])

63

In [5]:
chars = sorted(list(set(text)))
vocab_len = len(chars)
vocab_len

65

In [6]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s:[stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])
print(encode("hello"))
print(decode([46, 43, 50, 50, 53]))

[46, 43, 50, 50, 53]
hello


In [7]:
encoded_text = encode(text)
data = torch.tensor(encoded_text, dtype=torch.long)
data.shape

torch.Size([1115394])

In [72]:
n = int(0.9*len(data))
train = data[:n]
val = data[n:]

In [73]:
block_size = 8

In [87]:
torch.manual_seed(1337)
batch_size = 4

def get_batch(split, batch_size):
    data = train if split=='train' else val
    ix = torch.randint(0, len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [75]:
x_b, y_b = get_batch('train')

In [77]:
a = x_b.view(-1)
a.shape

torch.Size([32])

In [78]:
y_b.view(-1)

tensor([43, 58,  5, 57,  1, 46, 43, 39, 53, 56,  1, 58, 46, 39, 58,  1, 58,  1,
        58, 46, 39, 58,  1, 46, 17, 27, 10,  0, 21,  1, 54, 39])

In [80]:
model = BLModel(vocab_len)
logits, loss = model(x_b, y_b)
print(loss)

tensor(4.7288, grad_fn=<NllLossBackward0>)


In [85]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(idx, max_tokens=100)[0].tolist()))


LM,VPBBZslB3q;X-?XYjnnoS,KPxOhFXFVAB:$Ssy'KiuBH:iztcqsjOV? ypX.CV?E,IW!B3RrkQ3slxSJWyp
RxruO ?OmnCV?


In [90]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [102]:
batch_size = 32
epochs = 10000

for i in range(epochs):
    x, y = get_batch('train', batch_size)
    logits, loss = model(x, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print(loss.item())

2.54422664642334


In [103]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(idx, max_tokens=300)[0].tolist()))


DUSearspl woalsorthathay st sly ch hentrse,

'sur titest'es anaswh; s'drvisthe ANTI tshef mif thithonik y
ARDYer mit, t velags g I beffuror, tou IONTanoredie mabongrelash, buste sicod.

Medosewnof the offt, atingey y hear w, tontr hee ct meho myoulorey;
Antlondy wek thal,
Cony hath, win, wl d r's n 
